In [1]:
from twitter_credentials import *
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from time import time
import csv
import os

In [2]:
sid = SentimentIntensityAnalyzer()

# tweepy

In [3]:
import tweepy

In [4]:
auth = tweepy.OAuthHandler(twitter_credentials['API key'], twitter_credentials['API secret'])
auth.set_access_token(twitter_credentials['token'], twitter_credentials['token secret'])

api = tweepy.API(auth)

In [19]:
#override tweepy.StreamListener to add logic to on_status
class MyStreamListener(tweepy.StreamListener):
    
    limit = 0
    counter = 0
    data_file = 'data.csv'
    data_limit = 1e9

    def on_status(self, tweet):
        
        if hasattr(tweet, 'text'):

            if hasattr(tweet, 'retweeted_status'):
                tweet = tweet.retweeted_status
            
            lower = tweet.text.lower() # convert to lower case
            
            if 'sanders' in lower:
                fn = 'data_sanders.csv'
            elif 'clinton' in lower:
                fn = 'data_clinton.csv'
            elif 'trump' in lower:
                fn = 'data_trump.csv'
            elif 'cruz' in lower:
                fn = 'data_cruz.csv'
            else:
                fn = 'data_unknown.csv'

            with open(fn, 'ab') as csvfile:
                    spamwriter = csv.writer(csvfile, delimiter=' ',
                                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
                    spamwriter.writerow([time(), sid.polarity_scores(tweet.text)['compound']])

            self.counter += 1
            if self.limit and self.counter >= self.limit:
                self.counter = 0
                return False
#             file_size = os.path.getsize(self.data_file)
#             if file_size > self.data_limit:
#                 return False
        
    def on_error(self, status_code):
        print status_code
        if status_code == 420:
            #returning False in on_data disconnects the stream
            return False

In [26]:
myStreamListener = MyStreamListener()
myStreamListener.data_file = 'data.csv'
myStream = tweepy.Stream(auth = api.auth, listener=myStreamListener)
myStream.timeout = 60*5

In [22]:
myStream.disconnect()

In [ ]:
myStream.sample(languages=['en'])

In [ ]:
myStream.filter(languages=['en'], locations=[-124.848974, 24.396308, -66.885444, 49.384358], stall_warnings=True)

In [ ]:
myStream.filter(track=['Sanders, Clinton, Trump, Ted Cruz'], languages=['en'])